In [138]:
import pandas as pd
import numpy as np
from gurobipy import *
import random
import itertools
import time 


In [139]:

# Parameters
teams = ["Adelaide Crows", 
         "Brisbane Lions", 
         "Carlton Blues", 
         "Collingwood Magpies",
         "Essendon Bombers", 
         "Fremantle Dockers", 
         "Geelong Cats", 
         "Gold Coast Suns",
         "Greater Western Sydney Giants", 
         "Hawthorn Hawks", 
         "Melbourne Demons",
         "North Melbourne Kangaroos", 
         "Port Adelaide Power",
         "Richmond Tigers",
         "St Kilda Saints",
         "Sydney Swans",
         "West Coast Eagles",
         "Western Bulldogs"]

team_numbers = {team: number for number, team in enumerate(sorted(teams), start=0)}

locations = ['VIC','NSW','SA','WA','QLD']

location_numbers = {location: number for number, location in enumerate(sorted(locations), start=0)}

home_locations = ['SA','QLD','VIC','VIC','VIC','WA','VIC','QLD','NSW','VIC',
                          'VIC','VIC','SA','VIC','VIC','NSW','WA','VIC']

ranking = [10,2,5,1,11,14,12,15,7,16,4,17,3,13,6,8,18,9] # 2023 pre-finals rankings

team_fans = [60,45,93,102,79,61,81,20,33,72,68,50,60,100,58,63,100,55] # 2023 number of members, in 000's

wins = [11,17,13.5,18,11,10,10.5,9,13,7,16,3,17,10.5,13,12.5,3,12]

stadiums = ['MCG','Marvel','GMHBA','Adelaide Oval','Optus','Gabba','HBS','SCG','Giants']

stadium_numbers = {stadium: number for number, stadium in enumerate(sorted(stadiums), start=0)}

stadium_locations = ['VIC','VIC','VIC','SA','WA','QLD','QLD','NSW','NSW']

home_stadiums = [['Adelaide Oval'],['Gabba'],['MCG','Marvel'],['MCG','Marvel'],['MCG','Marvel'],['Optus'],['GMHBA'],
                 ['HBS'],['Giants'],['MCG'],['MCG'],['Marvel'],['Adelaide Oval'],
                 ['MCG'],['Marvel'],['SCG'],['Optus'],['Marvel']]

home_location_stadiums = [[] for i in range(len(teams))]
for i in range(len(teams)):
    for j in range(len(stadiums)):
        if stadium_locations[j] == home_locations[i]:
            home_location_stadiums[i].append(stadiums[j])

stadium_size = [100,53,40,54,60,39,27,47,24] # Stadium sizes in 000's

rivals = [['Port Adelaide Power'],
          ['Gold Coast Suns','Collingwood Magpies'], # From wikipedia
          ['Essendon Bombers','Richmond Tigers','Collingwood Magpies', 'Fremantle Dockers'],
          ['Carlton Blues','Essendon Bombers','Brisbane Lions','Melbourne Demons','Richmond Tigers','Geelong Cats',
           'Hawthorn Hawks', 'West Coast Eagles','Port Adelaide Power'],
          ['Carlton Blues','Collingwood Magpies','Richmond Tigers','Hawthorn Hawks','North Melbourne Kangaroos'],
          ['Carlton Blues','West Coast Eagles'],
          ['Collingwood Magpies','Hawthorn Hawks'],
          ['Brisbane Lions'],
          ['Sydney Swans','Western Bulldogs'],
          ['Hawthorn Hawks','Geelong Cats','Essendon Bombers', 'North Melbourne Kangaroos'],
          ['Collingwood Magpies'],
          ['Essendon Bombers','Hawthorn Hawks'],
          ['Adelaide Crows','Collingwood Magpies'],
          ['Carlton Blues','Collingwood Magpies','Essendon Bombers'],
          ['Sydney Swans'],
          ['West Coast Eagles','St Kilda Saints','Greater Western Sydney Giants','Hawthorn Hawks'],
          ['Fremantle Dockers','Sydney Swans','Collingwood Magpies'],
          ['Greater Western Sydney Giants']]

rivals_num = [[team_numbers[i] for i in rivals[j]] for j in range(len(rivals))]

timeslots = [i for i in range(7)]
timeslot_values = [10,13,5,6,11,5,4] # Change later according to attendances
timeslot_names = ['Thursday Night','Friday Night','Saturday Afternoon','Saturday Evening',
                  'Saturday Night','Sunday Afternoon','Sunday Evening']

Ts = range(len(teams))
Ss = range(len(stadiums))
timeslots = range(7)
rounds = range(22)

# Decision Variables
fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]

In [140]:
def feasibility(fixture):
    violated = 0
    critical = 0
    
    # Each team plays once a week
    tally = 0
    for i in Ts: 
        for r in rounds:
            tally += sum(sum(sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for j in Ts) for s in Ss) for t in timeslots) 
    if tally != 1:
        # print("Violated Critical Constraint: Each Team Plays Once a Week")
        critical += 1
            
    # Each team has eleven home games
    tally = 0
    for i in Ts:
        sum(sum(sum(sum(fixture[r][t][stadium_numbers[s]][j][i] for j in Ts) for s in home_stadiums[i]) for t in timeslots) for r in rounds) 
    if tally != 11:
        # print("Violated Critical Constraint: Eleven Home Games")
        critical += 1
    
    # Teams can't play themselves, and play all other teams once or twice (not twice away, or twice home)
    tally1 = 0
    tally2 = 0
    tally3 = 0
    for i in Ts:
        tally += sum(sum(sum(fixture[r][t][s][i][i] for s in Ss) for t in timeslots) for r in rounds) 
        for j in Ts:
            if i != j:
                # Cannot play more than twice 
                tally2 += sum(sum(sum(fixture[r][t][s][j][i] for s in Ss) for t in timeslots) for r in rounds) 
                # Play at least once 
                tally3 += sum(sum(sum(fixture[r][t][s][j][i] + fixture[r][t][s][i][j] for s in Ss) for t in timeslots) for r in rounds) 
    if tally1 > 0:
        critical += 1 
        # print("Violated Critical Constraint: Teams Cannot Play Themselves")     
    if tally2 > 1:
        # print("Violated Critical Constraint: Cannot Play More than Twice Per Round")   
        critical += 1
    if tally3 < 1:
        # print("Violated Critical Constraint: Each Team Plays at Least Once Per Round")   
        critical += 1
    
    # At least a five day break 
    tally = 0        
    for i in Ts:
        for r in rounds[:-1]:
            tally = tally + sum(sum(sum(fixture[r][t][s][j][i]+fixture[r][t][s][i][j] for j in Ts) for s in Ss) for t in [5,6]) + sum(sum(sum(fixture[r+1][t][s][j][i]+fixture[r+1][t][s][j][i] for j in Ts) for s in Ss) for t in [0])
    if 1 < tally:
        # print("Violated Constraint: At Least 5 Day Break")   
        violated += 1

    # No three games in a row outside home location
    tally = 0
    for i in Ts:
        for r in rounds[:-2]:
            tally += sum(sum(sum(sum(fixture[r_][t][stadium_numbers[s]][j][i]+fixture[r_][t][stadium_numbers[s]][i][j] for j in Ts) for s in home_location_stadiums[i]) for t in timeslots) for r_ in range(r,r+3)) 
    if tally < 1:
        # print("Violated Critical Constraint: No three games in a row outside home location")   
        violated += 1 

    # No four away games in a row
    tally = 0
    for i in Ts:
        for r in rounds[:-3]:
            tally += sum(sum(sum(sum(fixture[r_][t][s][j][i] for j in Ts) for s in Ss) for t in timeslots)
                                     for r_ in range(r,r+4)) 
    if tally < 1:
        # print("Violated Critical Constraint: No four away games in a row") 
        violated += 1  

    # No 2+ games in one day in the same stadium
    for r in rounds:
        tally1 = 0
        tally2 = 0
        tally3 = 0
        for s in Ss:
            # Saturday & Sunday Games 
            tally1 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for t in [5, 6]) 
            tally2 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for t in [2, 3, 4])
            # One game per timeslot per stadium 
            for t in [0,1]:
                tally3 += sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts)  
        if tally1 > 1:
            # print("Violated Critical Constraint: No 2+ games in one day in the same stadium (Sat, Sun)") 
            violated += 1
        if tally2 > 1:
            violated += 1
            # print("Violated Critical Constraint: No 2+ games in one day in the same stadium") 
        if tally3 > 1:
            # print("Violated Critical Constraint: One game per timeslot per stadium ") 
            violated += 1
    
    tally1 = 0
    tally2 = 0
    tally3 = 0
    tally4 = 0

    # No more than two games in any timeslot, and only one on Thursday and Friday night, incentivise games in each timeslot
    for r in rounds:
        
        # At least 2 sunday games 
        tally1 += sum(sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss) for t in [5,6])
        
        # One Thursday & Friday Night Game
        for t in [0,1]:
            tally2 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss)
        
        # No More Than Two Simultaneous Games
        for t in [2,3,4,5,6]:
            tally3 += sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss)     

        for t in timeslots:
            if sum(sum(sum(fixture[r][t][s][j][i] for i in Ts) for j in Ts) for s in Ss) < 1: 
                tally4 += 1
    
    if tally1 < 2:
        # print("Violated Critical Constraint: At least 2 sunday games ") 
        violated += 1
    if tally2 > 1: 
        # print("Violated Critical Constraint: One Thursday & Friday Night Game ") 
        violated += 1
    if tally3 > 2:
        # print("Violated Critical Constraint: No More Than Two Simultaneous Games ") 
        violated += 1
    if tally4 > 0:
        # print("Violated Critical Constraint: incentivise games in each timeslot") 
        violated += 1
    
    num_rivals = 0
    for r in rounds: 
        for t in timeslots:
            for i in Ts:
                for j in Ts:
                    for s in Ss:
                        if j in rivals[i]: 
                            num_rivals += 1
        if num_rivals < 0:
            # print("Violated Critical Constraint: No rivalry games") 
            critical += 1

    return violated, critical

In [141]:
# Attractiveness Function Parameters, adjust them as needed
alpha = 1.0
beta = 1.0
gamma = 1.0
sigma = 1.0
xi = 1.0

def attractiveness(i, j, s, t, r):
    score = 1
    if r == 0:
        score *= 4
    elif r == 1:
        score *= 2
    elif r == 21:
        score *= 1.5
    
    if j in [team_numbers[i] for i in rivals[0]]:
        score *= 1+alpha
    
    score /= np.sqrt(1+abs(ranking[i]-ranking[j]))
    score  /= np.sqrt(ranking[i]+ranking[j])
    
    if stadium_locations[s] == home_locations[j]:
        score *= (1+beta)
        
    score *= np.sqrt(stadium_size[s])
    score *= (team_fans[i]+0.5*team_fans[j])
    
    score *= timeslot_values[t]
    
    return score

In [142]:
def probability_win(i, j, s):
    probability = wins[i]/(wins[i]+wins[j])
    if stadiums[s] not in home_location_stadiums[j]:
        probability += (1-probability)/2.5
    elif stadiums[s] not in home_stadiums[j]:
        probability += (1-probability)/4
    else:
        probability += (1-probability)/10
        
    return probability

In [143]:
def expected_win_variance(fixture):
    results = []
    expected_wins = [0]*18
    for r in rounds:
        for i in Ts:
            for j in Ts:
                for s in Ss:
                    for t in timeslots:
                        expected_wins[i] += probability_win(i, j, s)*fixture[r][t][s][j][i]
                        expected_wins[i] += (1-probability_win(j, i, s))*fixture[r][t][s][i][j]
        
        results.append(np.var(expected_wins))
    
    # print(results)
    return sum((i+1)*results[i] for i in range(len(results)))

In [144]:
def expected_win_variance(fixture):
    results = []
    expected_wins = [0]*18
    for r in rounds:
        for i in Ts:
            for j in Ts:
                for s in Ss:
                    for t in timeslots:
                        expected_wins[i] += probability_win(i, j, s)*fixture[r][t][s][j][i]
                        expected_wins[i] += (1-probability_win(j, i, s))*fixture[r][t][s][i][j]
        
        results.append(np.var(expected_wins))
    
    # print(results)
    return sum((i+1)*results[i] for i in range(len(results)))

In [145]:
def fixture_attractiveness(fixture,max_value,violated_factor,critical_factor,equality_factor):
    total_score = 0
    
    for r in rounds:
        for t in timeslots:
            value = 0
            for i in Ts:
                for j in Ts:
                    for s in Ss:
                        value += attractiveness(i, j, s, t, r)*fixture[r][t][s][j][i]
            
            total_score += min(max_value,value)
    
    violated, critical = feasibility(fixture)
    equality = equality_factor*expected_win_variance(fixture)
    
    return total_score - violated_factor*violated - critical_factor*critical - equality


In [146]:
def objective_value(fixture):

    violated, critical = feasibility(fixture)

    violated_penalty = 10000
    critical_penalty = 1000000
    # for i in Ts:
    #     for j in Ts:
    #         for s in Ss:
    #             for r in rounds:
    #                 for t in timeslots:
    #                     print(attractiveness(i,j,s,t,r))

    objective_value = - violated_penalty*violated - critical_penalty*critical + fixture_attractiveness(fixture, 10000000, critical_penalty, violated_penalty, 1000) 

    return objective_value

In [147]:
def genesis(pop_size=5):
    print('Genesis')
    
    pop = []
    for i in range(pop_size):
        fixture_matrix = [[[[[0 for i in Ts] for j in Ts] for s in Ss] for t in timeslots] for r in rounds]       
        for r in rounds: 
            for t in timeslots:
                # Randomly choose two teams. Ensure there is at least one rivalry match. 
                if t==0:
                    selection_i = random.choice(Ts)
                    selection_j = random.choice(rivals_num[selection_i])
                    selection_stadium = random.choice(home_location_stadiums[selection_i])
                    selection_stadium = stadium_numbers[selection_stadium]
                    fixture_matrix[r][t][selection_stadium][selection_j][selection_i] = 1 
                    # print('Rivalry Round: ', teams[selection_i], ' vs. ', teams[selection_j])
                else: 
                    team_pairing = random.sample(Ts, 2)
                    selection_i = team_pairing[0]
                    selection_j = team_pairing[1]
                    selection_stadium = random.choice(home_location_stadiums[selection_i])
                    selection_stadium = stadium_numbers[selection_stadium]
                    fixture_matrix[r][t][selection_stadium][selection_j][selection_i] = 1 
                    # print(teams[selection_i], ' vs. ', teams[selection_j])    
            # print('\n')  
        pop.append([fixture_matrix, objective_value(fixture_matrix)])
    return pop

In [148]:
import random

def select_parents(population):
    """
    Select two individuals from a population for reproduction.
    
    Args:
    - population (list): A list of individuals, where each individual is represented as a tuple (individual_data, objective_value).
    
    Returns:
    - parent1 (tuple): The first parent selected.
    - parent2 (tuple): The second parent selected.
    """
    
    print('Romance')
    # Sort the population by objective value in descending order (higher is better)
    sorted_population = sorted(population, key=lambda x: x[1], reverse=True)

    # Calculate the number of elite individuals (top 25%)
    num_elite = len(sorted_population) // 4

    if num_elite > 1:
        # If there are enough elite individuals, select the first parent as an elite individual
        parent1 = random.choice(sorted_population[:num_elite])
    else:
        # If there are not enough elite individuals, choose both parents randomly
        parent1 = random.choice(sorted_population)
    
    # Select the second parent randomly from the entire population
    parent2 = random.choice(population)

    return parent1, parent2



In [149]:
def get_dimensions(lst):
    dimensions = []

    while isinstance(lst, list):
        dimensions.append(len(lst))
        lst = lst[0] if len(lst) > 0 else None

    return dimensions

In [150]:
def birth(parent1, parent2):
    print('Birth')
    # print(get_dimensions(parent1), get_dimensions(parent2))
    # Determine a random crossover point
    crossover_point = np.random.randint(0, len(parent1))  # Include 0, exclude len(parent1)

    # Create the first child by combining the first part of parent1 and the second part of parent2
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))

    # Create the second child by combining the first part of parent2 and the second part of parent1
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))

    return [child1, objective_value(child1)], [child2, objective_value(child2)]
    

In [151]:
def evolutionPlus():

    start_time = time.time()  # Record the start time

    # Parameters 
    pop_size = 5  # Adjust as needed
    solns = []
    num_gen = 0
    duration = 60 * 10

    pop = genesis(pop_size)  # Use the provided pop_size when calling genesis
    best_soln = max(pop, key=lambda x: x[1])
    while time.time() - start_time < duration:
        num_gen += 1
        # Select Parents
        parent1, parent2 = select_parents(pop)

        # Create Children
        child1, child2 = birth(parent1[0], parent2[0])
        pop.append(child1)
        pop.append(child2)
        
        # Death
        family = [parent1, parent2, child1, child2]
        # Find family member with the minimum objective value
        min_value = 1000000000000
        to_die_idx = None
        for i, member in enumerate(family):
            if member[1] < min_value:
                min_value = member[1]
                to_die_idx = i

        # Remove the element at to_die_idx
        if to_die_idx is not None:
            print('Death')
            pop.pop(to_die_idx)

        # Occassionally perform local optimisation

        # Occasionally perform immigration

        # Find the index of the element with the smallest second element
        max_index = 0
        max_value = pop[0][1]
        for i, soln in enumerate(pop):
            if soln[1] > max_value:
                max_value = soln[1]
                max_index = i

        # Extract the element with the smallest second element
        best_soln = pop[max_index]
        solns.append(best_soln)

    for i, soln in enumerate(pop):
        if soln[1] < max_value:
            max_index = i
    best_soln = pop[max_index]

    print('Solution: ', best_soln[0])
    print('Value: ', best_soln[1])

    return best_soln


In [152]:
best_soln = evolutionPlus()

Genesis


Romance
Birth
Death
Romance
Birth
Death
Romance
Birth
Death
Romance
Birth
Death
Romance
Birth
Death
Romance
Birth
Death
Solution:  [[[[[0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    ...
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]]

   [[0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    ...
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]]

   [[0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    ...
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]]

   ...

   [[0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    ...
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]]

   [[0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    ...
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]]

   [[0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    ...
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]
    [0 0 0 ... 0 0 0]]]


  [[[0 

In [153]:
best_fixture = best_soln[0]
for r in rounds: 
    print('\nRound ', r)
    for t in timeslots:
        for i in Ts:
            for j in Ts:
                for s in range(len(Ss)-1):
                    if best_fixture[r][t][s][j][i] == 1:
                        if j in [team_numbers[rival] for rival in rivals[i]]:
                            print("Rivalry Match! ", teams[i], " VS ", teams[j], ' AT ', stadiums[s], ' ON ', timeslot_names[t])
                        else: 
                            print(teams[i], " VS ", teams[j], ' AT ', stadiums[s], ' ON ', timeslot_names[t])


Round  0
Rivalry Match!  Essendon Bombers  VS  Carlton Blues  AT  HBS  ON  Thursday Night
Rivalry Match!  Gold Coast Suns  VS  Brisbane Lions  AT  Optus  ON  Friday Night
Gold Coast Suns  VS  West Coast Eagles  AT  Optus  ON  Saturday Afternoon
Fremantle Dockers  VS  Adelaide Crows  AT  SCG  ON  Saturday Evening
Rivalry Match! 

 Essendon Bombers  VS  Carlton Blues  AT  HBS  ON  Saturday Night
Port Adelaide Power  VS  Brisbane Lions  AT  MCG  ON  Sunday Afternoon
Melbourne Demons  VS  Brisbane Lions  AT  HBS  ON  Sunday Evening

Round  1
Rivalry Match!  Collingwood Magpies  VS  Brisbane Lions  AT  HBS  ON  Thursday Night
Rivalry Match!  Collingwood Magpies  VS  Richmond Tigers  AT  Marvel  ON  Saturday Afternoon
Fremantle Dockers  VS  Brisbane Lions  AT  SCG  ON  Saturday Evening
Melbourne Demons  VS  Hawthorn Hawks  AT  HBS  ON  Saturday Night
Adelaide Crows  VS  Gold Coast Suns  AT  MCG  ON  Sunday Afternoon

Round  2
Rivalry Match!  Gold Coast Suns  VS  Brisbane Lions  AT  Optus  ON  Thursday Night
Melbourne Demons  VS  Richmond Tigers  AT  HBS  ON  Friday Night
Richmond Tigers  VS  Fremantle Dockers  AT  Marvel  ON  Saturday Afternoon
Geelong Cats  VS  Melbourne Demons  AT  Marvel  ON  Saturday Evening
West Coast Eagles  VS  Brisbane Lions  AT  SCG  ON  Saturday Night
Geelong Cats  VS  Sydney Swans  AT  Ma

In [154]:
print(best_soln[1])

-73797472.5752212


In [155]:
feasibility(best_soln[0])

(69, 3)